In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
# nltk.download('stopwords') # Downloaded
# nltk.download('punkt') # Downloaded
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math



In [2]:
#We will load the default model which is english-core-web.

import spacy 


In [3]:
# use os.getcwd()to "get current working directory"
# Load wikipedia dataset

print("The directory we are workingin is {}".format(os.getcwd()))
database = pd.read_csv("topic_data.csv")

print(type(database))

The directory we are workingin is C:\Users\sumit
<class 'pandas.core.frame.DataFrame'>


In [4]:
# View inside the dataset

database

,Unnamed: 0,topic,abbreviation,wiki_title,wiki_summary,wiki_content,wiki_html,wiki_links,wiki_sections
0,0,acceptance criterion,NaN,NaN,Acceptance test–driven development (ATDD) is a...,Acceptance test–driven development (ATDD) is a...,"<div class=""mw-parser-output""><table class=""ve...",['A Guide to the Business Analysis Body of Kno...,Overview\n Creation \n Testing strategy \nAcce...
1,1,activation function,NaN,NaN,"In artificial neural networks, the activation ...","In artificial neural networks, the activation ...","<div class=""mw-parser-output""><div role=""note""...","['Action potential', 'Activating function', 'A...",Functions\n Alternative structures \n Comparis...
2,2,active constraint,NaN,NaN,"In mathematical optimization, a problem is def...","In mathematical optimization, a problem is def...","<div class=""mw-parser-output""><div role=""note""...","['Active set', 'Feasible region', 'Frank–Wolfe...",Active-set methods\nReferences\nBibliography
3,3,adaboost,NaN,NaN,"AdaBoost, short for Adaptive Boosting, is a ma...","AdaBoost, short for Adaptive Boosting, is a ma...","<div class=""mw-parser-output""><table class=""ve...","['Anomaly detection', 'ArXiv', 'Artificial neu...",Overview\n Training \n Weighting \nDerivation\...
4,4,adaline,NaN,NaN,The Age of Adaline is a 2015 American romantic...,The Age of Adaline is a 2015 American romantic...,"<div class=""mw-parser-output""><p class=""mw-emp...","['2015 Teen Choice Awards', ""42nd People's Cho...",Plot\nCast\nProduction\n Casting \n Filming \n...
...,...,...,...,...,...,...,...,...,...
775,775,wishart distribution soft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
776,776,within-class covariance soft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,777,woodbury identity soft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778,778,wrapped distribution soft,NaN,NaN,Wire wrap was invented to wire telephone cross...,Wire wrap was invented to wire telephone cross...,"<div class=""mw-parser-output""><div role=""note""...","['AWG', 'American wire gauge', 'Apollo Guidanc...",Overview\nApplication considerations\nHistory\...


In [5]:
# Preview the first 5 lines of the loaded dataset
database.head()

#calculate the length of the dataset
len(database)


780

In [6]:
# To check the types of each column in our database
database.dtypes

Unnamed: 0         int64
topic             object
abbreviation      object
wiki_title       float64
wiki_summary      object
wiki_content      object
wiki_html         object
wiki_links        object
wiki_sections     object
dtype: object

In [7]:
# Describing column of database with .describe()

#For numeric columns, describe() returns basic statistics: 
#the value count, mean, standard deviation, minimum, maximum, and 25th, 50th, and 75th 
#quantiles for the data in a column.

#For string columns, describe() returns the value count, the number of unique entries, 
#the most frequently occurring value (‘top’), and the number of times the top value occurs (‘freq’)

# describing wiki_content column in our database
database['wiki_content'].describe()


count                                                   689
unique                                                  483
top       Hidden Markov Model (HMM) is a statistical Mar...
freq                                                     17
Name: wiki_content, dtype: object

In [8]:
# Describing a full dataframe gives summary statistics for the numeric columns only, 
# and the return format is another DataFrame.
database.describe()

,Unnamed: 0,wiki_title
count,780.000000,0.0
mean,389.500000,NaN
std,225.310896,NaN
min,0.000000,NaN
25%,194.750000,NaN
50%,389.500000,NaN
75%,584.250000,NaN
max,779.000000,NaN


In [9]:
# Three primary methods for selecting columns from dataframes in pandas
# selecting columns

database.wiki_html.head() #or,

database.iloc[:,5].head() #or,

database['wiki_title'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: wiki_title, dtype: float64

In [10]:
# Explore the database and checkout the content in it
activation = database[database['topic']== 'activation function']


In [11]:
activation

,Unnamed: 0,topic,abbreviation,wiki_title,wiki_summary,wiki_content,wiki_html,wiki_links,wiki_sections
1,1,activation function,NaN,NaN,"In artificial neural networks, the activation ...","In artificial neural networks, the activation ...","<div class=""mw-parser-output""><div role=""note""...","['Action potential', 'Activating function', 'A...",Functions\n Alternative structures \n Comparis...


In [12]:
#About the variable

activation['wiki_content']

1    In artificial neural networks, the activation ...
Name: wiki_content, dtype: object

In [13]:
# Another example - Explore the database and checkout the content in it
adaline = database[database['topic'] == 'adaline']

# Another example - About the variable
adaline['wiki_content']


4    The Age of Adaline is a 2015 American romantic...
Name: wiki_content, dtype: object

In [14]:
title = "Wiki_Data" # Folder name where dataset is present
alpha = 0.3
# Taking all folders
folders = [x[0] for x in os.walk(str(os.getcwd())+'/'+title+'/')]
folders[0] = folders[0][:len(folders[0])-1]

In [15]:
# View the directory path

folders

['C:\\Users\\sumit/Wiki_Data']

In [16]:
N = len (database)
N

780

In [17]:
def print_doc(id):
    print(database[id])
    file = open(database[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)


In [18]:
# Preprocessing
def convert_lower_case(database):
    return np.char.lower(database)

In [19]:
def remove_stop_words(database):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(database))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [20]:
def remove_punctuation(database):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        database = np.char.replace(database, symbols[i], ' ')
        database = np.char.replace(database, "  ", " ")
    database = np.char.replace(database, ',', '')
    return database

In [21]:
def remove_apostrophe(database):
    return np.char.replace(database, "'", "")

In [22]:
def stemming(database):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(database))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [23]:
def convert_numbers(database):
    tokens = word_tokenize(str(database))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [29]:
def preprocess(database):
    database = convert_lower_case(database)
    database = remove_punctuation(database) #remove comma seperately
    database = remove_apostrophe(database)
    database = remove_stop_words(database)
    database = convert_numbers(database)
    database = stemming(database)
    database = remove_punctuation(database)
    database = convert_numbers(database)
    database = stemming(database) #needed again as we need to stem the words
    database = remove_punctuation(database) #needed again as num2word is giving few hypens and commas fourty-one
    database = remove_stop_words(database) #needed again as num2word is giving stop words 101 - one hundred and one
    

In [30]:
database


,Unnamed: 0,topic,abbreviation,wiki_title,wiki_summary,wiki_content,wiki_html,wiki_links,wiki_sections
0,0,acceptance criterion,NaN,NaN,Acceptance test–driven development (ATDD) is a...,Acceptance test–driven development (ATDD) is a...,"<div class=""mw-parser-output""><table class=""ve...",['A Guide to the Business Analysis Body of Kno...,Overview\n Creation \n Testing strategy \nAcce...
1,1,activation function,NaN,NaN,"In artificial neural networks, the activation ...","In artificial neural networks, the activation ...","<div class=""mw-parser-output""><div role=""note""...","['Action potential', 'Activating function', 'A...",Functions\n Alternative structures \n Comparis...
2,2,active constraint,NaN,NaN,"In mathematical optimization, a problem is def...","In mathematical optimization, a problem is def...","<div class=""mw-parser-output""><div role=""note""...","['Active set', 'Feasible region', 'Frank–Wolfe...",Active-set methods\nReferences\nBibliography
3,3,adaboost,NaN,NaN,"AdaBoost, short for Adaptive Boosting, is a ma...","AdaBoost, short for Adaptive Boosting, is a ma...","<div class=""mw-parser-output""><table class=""ve...","['Anomaly detection', 'ArXiv', 'Artificial neu...",Overview\n Training \n Weighting \nDerivation\...
4,4,adaline,NaN,NaN,The Age of Adaline is a 2015 American romantic...,The Age of Adaline is a 2015 American romantic...,"<div class=""mw-parser-output""><p class=""mw-emp...","['2015 Teen Choice Awards', ""42nd People's Cho...",Plot\nCast\nProduction\n Casting \n Filming \n...
...,...,...,...,...,...,...,...,...,...
775,775,wishart distribution soft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
776,776,within-class covariance soft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
777,777,woodbury identity soft,NaN,NaN,NaN,NaN,NaN,NaN,NaN
778,778,wrapped distribution soft,NaN,NaN,Wire wrap was invented to wire telephone cross...,Wire wrap was invented to wire telephone cross...,"<div class=""mw-parser-output""><div role=""note""...","['AWG', 'American wire gauge', 'Apollo Guidanc...",Overview\nApplication considerations\nHistory\...


In [35]:
# Extracting Database
processed_text = []
processed_title = []

for i in database[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))

In [36]:
# Calculating DF for all words
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

IndexError: list index out of range

In [ ]:
# DF
total_vocab_size = len(DF)

In [ ]:
total_vocab_size

In [ ]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [ ]:
# Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [ ]:

# tf_idf

# Calculating TF-IDF for Title
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [ ]:
# Merging the TF-IDF according to weights
for i in tf_idf:
    tf_idf[i] *= alpha

In [ ]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [37]:
len(tf_idf)

NameError: name 'tf_idf' is not defined

In [ ]:
# TF-IDF Matching Score Ranking
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)
    

matching_score(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")


In [ ]:
# print_doc(2)
# TF-IDF Cosine Similarity Ranking
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
# Vectorising tf-idf
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [ ]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [ ]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)

#     for i in out:
#         print(i, dataset[i][0])

Q = cosine_similarity(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")

In [ ]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)

#     for i in out:
#         print(i, dataset[i][0])

Q = cosine_similarity(10, "Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")